In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/BT4222_Group21

/content/drive/MyDrive/BT4222_Group21


In [3]:
!pip install PyPDF2
!pip install ntlk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [4]:
import PyPDF2

def pdfToText(pdfFilePath, outputFilePath):
    print(f"Processing {pdfFilePath}")
    # Open the PDF file
    pdf_file = open(pdfFilePath, 'rb')

    # Create a PDF reader object
    reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF file
    num_pages = len(reader.pages)

    # Initialize a string variable to hold the text
    text = ""

    # Iterate through all the pages and extract text
    for page_num in range(num_pages):
        page_obj = reader.pages[page_num]
        text += page_obj.extract_text()

    # Close the PDF file
    pdf_file.close()

    # Open the file in write mode
    with open(outputFilePath, 'w') as f:
        # Write the text to the file
        f.write(text)

    # Print a message to confirm that the file has been saved
    print(f"Text saved to {outputFilePath}.")

In [5]:
import unicodedata
def remove_non_english_chars(text):
    # normalize text and check if ASCII value more than 128 (non-english words)
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn' and
                   ord(c) < 128)

In [21]:
import os
import re
import nltk
import pandas as pd
nltk.download('punkt')

if not os.path.exists("pdf_text"):
    os.makedirs("pdf_text")


input_filename = '/content/drive/MyDrive/data/2024_released_CGN_ESG_report.pdf'
output_filename = '/content/drive/MyDrive/BT4222_Group21/pdf_text/CGN_2023.csv'
pdfToText(input_filename, output_filename)

# read the text file
with open(output_filename, 'r') as f:
        text = f.read()

# clean the data
text = remove_non_english_chars(text)
text = re.sub(r'\n', ' ', text)  # remove line breaks
text = re.sub(r'\s+', ' ', text)  # remove multiple spaces
sentences = nltk.sent_tokenize(text) # split the text into sentences
processed_sentences = []
for sentence in sentences:
    sentence = re.sub(r"\d+\n", "", sentence) # Remove page number
    sentence = re.sub(r"\n", " ", sentence) # Remove line breaks
    sentence = re.sub(r"\.(\s*\.){1,}", ". ", sentence) # Remove consecutive periods
    split_sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)(?<!\d\.)\s*(?=[A-Z])", sentence) # split text into sentences

    split_sentences = [s for s in split_sentences if s != "" and s != "."] # do not include empty sentences or "."
    processed_sentences.extend(split_sentences)

    # create a dataframe
df = pd.DataFrame({'sentences': processed_sentences})
df.to_csv(output_filename, index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing /content/drive/MyDrive/data/2024_released_CGN_ESG_report.pdf
Text saved to /content/drive/MyDrive/BT4222_Group21/pdf_text/CGN_2023.csv.


In [8]:
!pip install tensorflow
!pip install pytorch
!pip install torch
!pip install transformers

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


### ESG-BERT
- Domain Specific BERT Model for Text Mining in Sustainable Investing
- URL: https://huggingface.co/nbroad/ESG-BERT
- This pre-trained model is able to classify each text and returns a label number which correlates to a textual label:
    * __label__Business_Ethics :  0
    * __label__Data_Security :  1
    * __label__Access_And_Affordability :  2
    * __label__Business_Model_Resilience :  3
    * __label__Competitive_Behavior :  4
    * __label__Critical_Incident_Risk_Management :  5
    * __label__Customer_Welfare :  6
    * __label__Director_Removal :  7
    * __label__Employee_Engagement_Inclusion_And_Diversity :  8
    * __label__Employee_Health_And_Safety :  9
    * __label__Human_Rights_And_Community_Relations :  10
    * __label__Labor_Practices :  11
    * __label__Management_Of_Legal_And_Regulatory_Framework :  12
    * __label__Physical_Impacts_Of_Climate_Change :  13
    * __label__Product_Quality_And_Safety :  14
    * __label__Product_Design_And_Lifecycle_Management :  15
    * __label__Selling_Practices_And_Product_Labeling :  16
    * __label__Supply_Chain_Management :  17
    * __label__Systemic_Risk_Management :  18
    * __label__Waste_And_Hazardous_Materials_Management :  19
    * __label__Water_And_Wastewater_Management :  20
    * __label__Air_Quality :  21
    * __label__Customer_Privacy :  22
    * __label__Ecological_Impacts :  23
    * __label__Energy_Management :  24
    * __label__GHG_Emissions :  25


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import math

tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")
model = AutoModelForSequenceClassification.from_pretrained("nbroad/ESG-BERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

### Step 1: Mapping ESG-BERT's topics to S&P ESG Criteria Topics
1. Using ESG-BERT to classify our S&P Global ESG Criteria Topics into its labels
2. We only take the predict label with highest probability.
3. For those labels without any S&P ESG Criteria Topics, we will manually map them ourselves based on relevancy

`labels_to_criteria`
- Key: ESG-BERT's ESG_Labels
- Value: S&P Global ESG Criteria Topics

In [10]:
import pandas as pd
import numpy as np
labels_to_criteria = {}
manual_mapping = []

# Criteria_Topics is a concatenated list of criteria topics from the industries that we are analyzing
with open("../BT4222_Group21/Scrapers/criteria_topics.txt", "r") as f:
    criteria_topics = eval(f.read())

for topic in criteria_topics:
    inputs = tokenizer(topic, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    top_probs, top_labels = torch.topk(probs, k=1)
    label = model.config.id2label[top_labels[0][0].item()]
    prob = top_probs[0][0].item()

    if label not in labels_to_criteria:
        labels_to_criteria[label] = []
    labels_to_criteria[label].append(topic)

# KEY: ESG-BERT's ESG_Labels
# Value: List of S&P Global ESG Criteria Topics
labels_to_criteria

{'Physical_Impacts_Of_Climate_Change': ['Climate Strategy'],
 'Business_Model_Resilience': ['Human Capital Development',
  'Innovation Management'],
 'Human_Rights_And_Community_Relations': ['Human Rights',
  'Environmental Policy & Management Systems',
  'Social Impacts on Communities',
  'Stakeholder Engagement'],
 'Data_Security': ['Information Security Cybersecurity & System Availability',
  'Network Reliability'],
 'Energy_Management': ['Operational Eco-Efficiency',
  'Low Carbon Strategy',
  'Energy Mix',
  'Electricity Generation',
  'Fuel Efficiency'],
 'Product_Design_And_Lifecycle_Management': ['Product Stewardship',
  'Packaging',
  'Sustainable Marketing & Brand Perception',
  'Sustainable Finance',
  'Circular Fashion'],
 'Supply_Chain_Management': ['Supply Chain Management'],
 'Employee_Engagement_Inclusion_And_Diversity': ['Talent Attraction & Retention',
  'Financial Inclusion'],
 'Labor_Practices': ['Labor Practice Indicators'],
 'Systemic_Risk_Management': ['Risk & Cr

For those ESG_labels that are not mapped to any ESG criteria topics, we would map them manually based on relevancy

In [11]:
labels_to_criteria = {
    'Physical_Impacts_Of_Climate_Change':['Climate Strategy'],
    'Business_Model_Resilience': ['Human Capital Development', 'Innovation Management'],
    'Human_Rights_And_Community_Relations': ['Human Rights', 'Environmental Policy & Management Systems', 'Social Impacts on Communities', 'Stakeholder Engagement'],
    'Data_Security': ['Information Security Cybersecurity & System Availability', 'Network Reliability'],
    'Energy_Management': ['Operational Eco-Efficiency', 'Low Carbon Strategy', 'Energy Mix', 'Electricity Generation', 'Fuel Efficiency'],
    'Product_Design_And_Lifecycle_Management': ['Product Stewardship', 'Packaging', 'Sustainable Marketing & Brand Perception', 'Sustainable Finance', 'Circular Fashion'],
    'Supply_Chain_Management': ['Supply Chain Management'],
    'Employee_Engagement_Inclusion_And_Diversity': ['Talent Attraction & Retention', 'Financial Inclusion'],
    'Labor_Practices': ['Labor Practice Indicators'],
    'Systemic_Risk_Management': ['Risk & Crisis Management', 'Decarbonization Strategy'],
    'Product_Quality_And_Safety': ['Product Quality & Recall Management'],
    'Management_Of_Legal_And_Regulatory_Framework': ['Responsibility of Content', 'Compliance with Applicable Export Control Regimes'],
    'Competitive_Behavior': ['Corporate Governance', 'Market Opportunities'],
    'Employee_Health_And_Safety': ['Occupational Health & Safety', 'Health Outcome Contribution'],
    'Access_And_Affordability': ['Access to Healthcare'],
    'Business_Ethics': ['Business Ethics'],
    'Ecological_Impacts': ['Biodiversity', 'Sustainable Agricultural Practices'],
    'Waste_And_Hazardous_Materials_Management': ['Food Loss & Waste'],
    'Water_And_Wastewater_Management': ['Water Related Risks'],
    "Air_Quality": ['Low Carbon Strategy'], # manual mapping
    "Critical_Incident_Risk_Management": ['Risk & Crisis Management'], # manual mapping
    "Customer_Privacy": ['Information Security Cybersecurity & System Availability'], # manual mapping
    "Customer_Welfare": ['Social Impacts on Communities', 'Human Rights', 'Stakeholder Engagement'], # manual mapping
    "Director_Removal": ['Business Ethics'], # manual mapping
    "GHG_Emissions": ['Operational Eco-Efficiency', 'Low Carbon Strategy', 'Decarbonization Strategy'], # manual mapping
    "Selling_Practices_And_Product_Labeling": ['Business Ethics', 'Product Stewardship', 'Packaging', 'Sustainable Marketing & Brand Perception'], # manual mapping
}

### Step 2: Preparing all the mappings collected from webscrapping

Import Industry to Criteria Topics Mapping as `industry_criteria_mapping`
- Key: Industry
- Value: List of most relevant criteria topics

In [12]:
industry_criteria_df = pd.read_csv("../BT4222_Group21/Scrapers/Industry Criteria Topics.csv").transpose()
industry_criteria_mapping = {}

for industry, topics in industry_criteria_df.iterrows():
    if industry not in industry_criteria_mapping:
        industry_criteria_mapping[industry] = []
    topics = topics.dropna()
    industry_criteria_mapping[industry].extend(topics.to_list())
industry_criteria_mapping

{'Industry: THQ Computers & Peripherals and Office Electronics': ['Climate Strategy',
  'Human Capital Development',
  'Human Rights',
  'Information Security Cybersecurity & System Availability',
  'Innovation Management',
  'Operational Eco-Efficiency',
  'Product Stewardship',
  'Supply Chain Management'],
 'Industry: SOF Software': ['Climate Strategy',
  'Environmental Policy & Management Systems',
  'Human Capital Development',
  'Information Security Cybersecurity & System Availability',
  'Innovation Management',
  'Operational Eco-Efficiency',
  'Talent Attraction & Retention'],
 'Industry: RTS Retailing': ['Climate Strategy',
  'Human Rights',
  'Labor Practice Indicators',
  'Operational Eco-Efficiency',
  'Packaging',
  'Risk & Crisis Management',
  'Supply Chain Management',
  'Sustainable Marketing & Brand Perception'],
 'Industry: SEM Semiconductors & Semiconductor Equipment': ['Climate Strategy',
  'Environmental Policy & Management Systems',
  'Human Capital Development

Import the company to industry mappings as `company_industry_mapping` dictionary
- key: TICKER
- Value: Industry

In [13]:
with open("../BT4222_Group21/Scrapers/company_industry_mapping.txt", "r") as f:
    company_industry_mapping = eval(f.read())

company_industry_mapping

{'AAPL': 'Industry: THQ Computers & Peripherals and Office Electronics',
 'MSFT': 'Industry: SOF Software',
 'ADBE': 'Industry: SOF Software',
 'AMZN': 'Industry: RTS Retailing',
 'HD': 'Industry: RTS Retailing',
 'NVDA': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'AVGO': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'TXN': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'AMD': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'QCOM': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'INTC': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'GOOGL': 'Industry: IMS Interactive Media, Services & Home Entertainment',
 'META': 'Industry: IMS Interactive Media, Services & Home Entertainment',
 'TSLA': 'Industry: AUT Automobiles',
 'UNH': 'Industry: HEA Health Care Providers & Services',
 'XOM': 'Industry: OGX Oil & Gas Upstream & Integrated',
 'CVX': 'Industry: OGX Oil & Gas Upstream & Integrated',
 'JPM': 'Industry: BNK Ban

### Step 3: Run report into ESG-BERT model to classify each sentence

In [14]:
import torch
import math

def run_model(input_file, model):
    counter = 1
    max_seq_length = tokenizer.model_max_length
    # with open(input_file, "r") as f:
    input_df = pd.read_csv(input_file)
    output = {}
    for line in input_df['sentences']:
        if len(line) <= max_seq_length:
            output['Line ' + str(counter)] = {}
            inputs = tokenizer(line, return_tensors="pt")
            outputs = model(**inputs)
            probs = outputs.logits.softmax(dim=1)

            # Extract the top 3 probabilities and labels
            top_probs, top_labels = torch.topk(probs, k=3)
            # store output
            output['Line ' + str(counter)]['Sentence'] = line
            for i in range(3):
                label = model.config.id2label[top_labels[0][i].item()]
                prob = top_probs[0][i].item()
                output['Line ' + str(counter)][f'ESG BERT Topic {i+1}'] = (label, prob)
            counter += 1
        else:
            # split input text into chunks
            text_chunks = []
            for i in range(math.ceil(len(line)/max_seq_length)):
                start = i * max_seq_length
                end = min((i+1)*max_seq_length, len(line))
                text_chunks.append(line[start:end])
            for chunk in text_chunks:
                output['Line ' + str(counter)] = {}
                inputs = tokenizer(chunk, return_tensors="pt", padding= True, truncation=True, max_length=max_seq_length)
                outputs = model(**inputs)
                probs = outputs.logits.softmax(dim=1)
                top_probs, top_labels = torch.topk(probs, k=3)

                output['Line ' + str(counter)]['Sentence'] = line
                for i in range(3):
                    label = model.config.id2label[top_labels[0][i].item()]
                    prob = top_probs[0][i].item()
                    output['Line ' + str(counter)][f'ESG BERT Topic {i+1}'] = (label, prob)
                counter += 1
    return output

### Step 4: Mapping output labels to our criteria topics
- We will map the output labels generated by ESG-BERT with the mapping, `mapped_to_criteria`
- We also have decided to classify sentences whose label with the highest probability is less than 0.50 as `Non-ESG`

In [24]:
''' [How it works]
    1. Get the list of most relevant criteria topics for the company based on its industry
    2. Based on the ESG_label with the highest probability, we get the list of potential criteria topics that can be mapped to the target ESG_label
    3. We check the list of potential criteria topics using the list of most relevant criteria topics
    4. If it exists, we just map the label to that criteria topic.
'''
def process_output(result, industry_criteria_mapping):
    # 将行业硬编码为“新能源”
    industry = "New Energy"

    # 初始化 DataFrame
    df = pd.DataFrame.from_dict(result, orient='index')

    # 获取“新能源”行业的相关 ESG 主题列表
    industry_criteria_topics = industry_criteria_mapping[industry]

    def mapperFunction(row, industry_criteria_topics):
        value = row[1]
        if value < 0.5:
            return ("NON-ESG", value)
        else:
            # 获取当前 ESG 标签的潜在映射主题
            mapped_criteria_topics = labels_to_criteria[row[0]]
            result = []
            # 检查是否符合“新能源”行业标准主题
            for topic in mapped_criteria_topics:
                if topic in industry_criteria_topics:
                    result.append(topic)
            return (result, value)
    df['Mapped Criteria Topic 1'] = df['ESG BERT Topic 1'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    df['Mapped Criteria Topic 2'] = df['ESG BERT Topic 2'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    df['Mapped Criteria Topic 3'] = df['ESG BERT Topic 3'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    return df

In [23]:
industry

'Industry: TRA Transportation and Transportation Infrastructure'

### Step 5: Export as CSV

In [16]:
def export_output_to_csv(df, output_filename):
    df.to_csv(output_filename)

### Step 6: Run all reports into the ESG-BERT Model

### NOTE: This will take about 2 hours to classify all 50 ESG reports
- For sample output, please run the commented code instead and comment out this chunk.

In [26]:
import os
import pandas as pd

'''
Note: This will take about 2 hours to classify all 25 ESG reports.

For a sample output please run the code chunk below
'''
input_file = '../BT4222_Group21/pdf_text/CGN_2023.csv'
output_file = "../BT4222_Group21/output/classified_sentences/AAPL_2022.csv"

if not os.path.exists(output_path):
    os.makedirs(output_path)
output_result = run_model(input_file, model)
output_df = process_output(output_result, industry_criteria_mapping)
export_output_to_csv(output_df, output_file)
print(f"Exported to {output_file}")

######## SAMPLE OUTPUT RUN THIS ########
# filename = "AMZN_2021.csv"
# ticker = filename.split("_")[0]
# industry = company_industry_mapping[ticker]
# input_file = "pdf_text" + "/" + filename
# ouptut_file = "output/classified_sentences/" + filename.replace(".csv", "_results.csv")
# output_result = run_model(input_file, model)
# output_df = process_output(output_result, industry_criteria_mapping, industry)
# export_output_to_csv(output_df, output_file)

KeyError: 'New Energy'